# MSAF examples using 25 songs from my FotF dataset

In [1]:
from __future__ import print_function
import msaf
import os
import librosa.display
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import IPython.display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


ModuleNotFoundError: No module named 'msaf'

# Select DS_input folder, graph output folder, algorithm & params

In [1]:
ds_path = '/home/shared/cuzokwe/datasets/FotF/audio'
single_file_path = '/home/shared/cuzokwe/datasets/FotF/audio/Around__the_World.flac'
out_path = '/home/shared/cuzokwe/datasets/FotF/visualizations'
b_alg = "olda"
b_algs = ['olda', 'scluster', 'foote', 'sf', 'cnmf']
l_alg = "fmc2d"
l_algs = ['fmc2d', 'scluster','cnmf']

# Display dataset files & algorithms w/ parameters

In [ ]:
titles = []
for file in os.listdir(ds_path):
    print(file)
    titles.append(file)
    
print("\nSegmenters: ", msaf.get_all_boundary_algorithms())
print("\nLabelers ", msaf.get_all_label_algorithms())

msaf.algorithms.vmo.config["vmo_k"] = 4
print(msaf.algorithms.vmo.config)


# Single file segment/display + boundary sonified audio

In [2]:
sonify = "test.wav"
boundaries, labels = msaf.process(single_file_path, boundaries_id=b_alg, labels_id=l_alg, sonify_bounds=True, out_bounds=sonify, out_sr=5512)

y, sr = librosa.load(single_file_path, sr=5512)
plt.figure()
figure(num=None, figsize=(20, 4), facecolor='w', edgecolor='k')
librosa.display.waveplot(y,sr=sr)

iter = 0
for label in labels:
    plt.axvline(x=boundaries[iter], ymin=0, ymax=1, color="orange", linestyle="--")
    plt.text(boundaries[iter] + 0.2, 0, label, fontsize=12)
    iter += 1
    
plt.title(os.path.basename(single_file_path) + " | " +"boundaries: " + b_alg + ", labels: " + l_alg + " | ")
plt.show()

out_audio = librosa.load(sonify, sr=44100)[0]
IPython.display.Audio(out_audio, rate=44100)

NameError: name 'msaf' is not defined

# Run segmentation over all songs -- store results in out_path

In [ ]:
for b_alg in b_algs:
    for l_alg in l_algs:
        for file in os.listdir(ds_path):
            if os.path.exists(out_path + "/" + b_alg + "-" + l_alg + "/" + file + ".png") == False:

                audio_path = ds_path + '/' + file

                boundaries, labels = msaf.process(audio_path, boundaries_id=b_alg, labels_id=l_alg)

                y, sr = librosa.load(audio_path, sr=5512)
                plt.figure()
                figure(num=None, figsize=(20, 4), facecolor='w', edgecolor='k')
                librosa.display.waveplot(y,sr=sr)

                # plot to graph
                iter = 0
                for label in labels:
                    plt.axvline(x=boundaries[iter], ymin=0, ymax=1, color="orange", linestyle="--")
                    plt.text(boundaries[iter] + 0.2, 0, label, fontsize=12)
                    iter += 1

                # try to save the figure
                print("Printing " + file)
                try:
                    plt.title(file + " | " +"boundaries: " + b_alg + ", labels: " + l_alg + " | ")

                    #runs are stored in [boundary algorithm]-[label algorithm] folders. check to see if folder has been created5
                    if not os.path.exists(out_path + "/" + b_alg + "-" + l_alg):
                        os.mkdir(out_path + "/" + b_alg + "-" + l_alg)
                        print("New boundary-label pair. Creating new directory...")
                        plt.savefig(out_path + "/" + b_alg + "-" + l_alg + "/" + file + ".png")
                    else:
                        plt.savefig(out_path + "/" + b_alg + "-" + l_alg + "/" + file + ".png")
                    plt.close()
                    print(file + " print complete")

                except:
                    print("Print failed for " + file + ". Try reducing sample rate to reduce total number of samples plotted.")

            else:
                print(file + ".png already exists here.")

In [ ]:
def comparevis(b_alg, l_alg):
    out_path = '/home/shared/cuzokwe/datasets/FotF/visualizations'
    title = 'Face_to_Face.flac'
    IPython.display.Image(filename = out_path + "/" + b_alg + "-" + l_alg + "/" + title + ".png")   
    file = open(out_path + "/" + b_alg + "-" + l_alg + "/" + title + ".png", "rb")
    image = file.read()
    widgets.Image(
        value=image,
        format='png',
        width= 1200,
        height=400,
    )

In [ ]:
interact(comparevis, b_alg = ['olda', 'scluster', 'foote', 'sf', 'cnmf'], l_alg = ['fmc2d', 'scluster','cnmf']);

In [ ]:
b_alg = 'scluster'
l_alg = 'fmc2d'
IPython.display.Image(filename = out_path + "/" + b_alg + "-" + l_alg + "/" + title + ".png") 

In [ ]:
b_alg = 'scluster'
l_alg = 'scluster'
IPython.display.Image(filename = out_path + "/" + b_alg + "-" + l_alg + "/" + title + ".png")  

In [ ]:
b_alg = 'scluster'
l_alg = 'cnmf'
IPython.display.Image(filename = out_path + "/" + b_alg + "-" + l_alg + "/" + title + ".png")

#### Current Notes/Observations
I see a lot of consecutive segmenting across multiple algorithms -- could be worth creating a system to concatenate those

boundary labels are almost never bar or beat synchronous -- might be possible to introduce a measure of boundary shift based on params


#### Use case ideas
Navigate through a collection of songs using segment level information to make segment decisions
